*Пренесување човек, волк, коза и зелка преку река*

In [17]:
import itertools
from collections import deque

In [5]:
passenger = ['Farmer', 'Goat', 'Cabbage', 'Wolf']
sailors = ['Farmer']

In [6]:
def unpack_state(state):
    boat = state[0]
    left_bank = set([passenger[index] for index, side in enumerate(state[1:]) if side == 'left'])
    right_bank = set([passenger[index] for index, side in enumerate(state[1:]) if side == 'right'])
    
    return boat, left_bank, right_bank

In [45]:
def obtain_boat_trips(available_passengers):
    boat_trips = [(passenger,) for passenger in available_passengers if passenger in sailors] #se dodava poseben tuple so farmer, zoso i toa e edno patuvanje koga e sam na brod
    for possible_boat_trip in itertools.combinations(available_passengers, 2):
        sailor_on_boat = any([passenger in sailors for passenger in possible_boat_trip])
        if sailor_on_boat:
            boat_trips.append((possible_boat_trip))

    return boat_trips

In [12]:
def permitted_river_bank(river_bank):
    if 'Wolf' in river_bank and 'Goat' in river_bank and 'Farmer' not in river_bank:
        return False
    if 'Goat' in river_bank and 'Cabbage' in river_bank and 'Farmer' not in river_bank:
        return False
    return True

In [13]:
def belongs(passenger, left_side, right_side):
    if passenger in left_side:
        return 'left'
    if passenger in right_side:
        return 'right'

In [19]:
def other_side(boat):
    if boat == 'left':
        return 'right'
    else:
        return 'left'

In [27]:
def expand_state(state):
    boat, left_bank, right_bank = unpack_state(state)
    
    if boat == 'left':
        boat_trips = obtain_boat_trips(left_bank)
    else:
        boat_trips = obtain_boat_trips(right_bank)
    
    new_states = []
    for boat_trip in boat_trips:
        if boat == 'left':
            new_left_bank = left_bank - set(boat_trip)
            new_right_bank = right_bank | set(boat_trip)
        if boat == 'right':
            new_left_bank = left_bank | set(boat_trip)
            new_right_bank = right_bank -set(boat_trip)
    
        if permitted_river_bank(new_left_bank) and permitted_river_bank(new_right_bank):
            farmer = belongs('Farmer', new_left_bank, new_right_bank)
            goat = belongs('Goat', new_left_bank, new_right_bank)
            cabbage = belongs('Cabbage', new_left_bank, new_right_bank)
            wolf = belongs('Wolf', new_left_bank, new_right_bank)
            new_states.append((other_side(boat), farmer, goat, cabbage, wolf))
            
    return new_states

In [2]:
def search_path(initial_state, goal_state):
    visited = {initial_state}
    states_queue = deque([[initial_state]])
    while states_queue:
        states_list = states_queue.popleft()
        state_to_expand = states_list[-1]
        for next_state in expand_state(state_to_expand):
            if next_state not in visited:
                if next_state == goal_state:
                    return states_list + [next_state]
                visited.add(next_state)
                states_queue.append(states_list + [next_state])
    return []

In [28]:
def visualise(path):
    if not path:
        print('Search path did not find a solution')
        return
    for pair_of_states in zip(path, path[1:]):
        boat_old, left_old, right_old = unpack_state(pair_of_states[0])
        boat_new, left_new, right_new = unpack_state(pair_of_states[1])
        delimiter_space = ' ' * 50
        separated_print(left_old)
        print(delimiter_space, end='')
        separated_print(right_old)
        print()
        if boat_old == 'left':
            delimiter = ' ' * 5 + '>' * 15 + ' ' * 5
            separated_print(left_new)
            print(delimiter, end='')
            separated_print(left_old - left_new)
            print(delimiter, end='')
            separated_print(right_old)
            print()
        else:
            delimiter = ' ' * 5 + '<' * 15 + ' ' * 5
            separated_print(left_old)
            print(delimiter, end='')
            separated_print(right_old - right_new)
            print(delimiter, end='')
            separated_print(right_new)
            print()
        separated_print(left_new)
        print(delimiter_space, end='')
        separated_print(right_new)
        print()
        print()
        print()

In [29]:
def separated_print(iterable):
    for element in iterable:
        print(element, end=' ')
    if not iterable:
        print('Empty', end='')

In [44]:
initial_state = ('left', 'left', 'left', 'left', 'left') # prvoto left e za pozicija na brodot
goal_state = ('right', 'right', 'right', 'right', 'right')
path = search_path(initial_state, goal_state)
#print(path)
visualise(path)

Farmer Cabbage Wolf Goat                                                   Empty
Cabbage Wolf      >>>>>>>>>>>>>>>     Farmer Goat      >>>>>>>>>>>>>>>     Empty
Cabbage Wolf                                                   Farmer Goat 


Cabbage Wolf                                                   Farmer Goat 
Cabbage Wolf      <<<<<<<<<<<<<<<     Farmer      <<<<<<<<<<<<<<<     Goat 
Farmer Cabbage Wolf                                                   Goat 


Farmer Cabbage Wolf                                                   Goat 
Wolf      >>>>>>>>>>>>>>>     Farmer Cabbage      >>>>>>>>>>>>>>>     Goat 
Wolf                                                   Farmer Cabbage Goat 


Wolf                                                   Farmer Cabbage Goat 
Wolf      <<<<<<<<<<<<<<<     Farmer Goat      <<<<<<<<<<<<<<<     Cabbage 
Farmer Wolf Goat                                                   Cabbage 


Farmer Wolf Goat                                                   Cab